In [148]:
import warnings
# 경고 무시하기
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from scipy import stats

import tensorflow as tf 
from keras.models import Model, Sequential
from keras.layers import Input, Dense

from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, make_scorer, mean_squared_error, mean_absolute_error
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, HuberRegressor, Lasso
from xgboost import XGBRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor

# SimpleImputer는 이 라인을 유지합니다
from sklearn.impute import SimpleImputer


In [117]:
## feature_engineering

train_df = pd.read_csv("./csv_file/Q1_train.csv")
test_df = pd.read_csv("./csv_file/Q1_train.csv")

In [118]:
labels = train_df['uenomax']
data = pd.concat([train_df, test_df], ignore_index = True)
data = data.drop("uenomax", 1)
ids = test_df['ru_id']

In [119]:
data["datetime"] = data["datetime"].apply(pd.to_datetime)

data["year"] = data["datetime"].dt.year
data["month"] = data["datetime"].dt.month
data["day"] = data["datetime"].dt.day
data["hour"] = data["datetime"].dt.hour
data["minute"] = data["datetime"].dt.minute

In [120]:
encoder = LabelEncoder()
train_df['ru_id_encoder'] = encoder.fit_transform(train_df['ru_id'])
train_df['ru_id_encoder'] = encoder.fit_transform(train_df['ru_id'])

In [121]:
data.drop( ['datetime','ru_id','erabaddatt', 'endcaddatt', 'endcmodbymenbatt', 'endcmodbysgnbatt', 'connestabatt', 'handoveratt', 'reestabatt', 'rachpreamblea'], axis=1, inplace=True)

In [122]:
data.isna().sum()

scgfail                                0
scgfailratio                           0
erabaddsucc                            0
endcaddsucc                            0
endcmodbymenbsucc                      0
endcmodbysgnbsucc                      0
connestabsucc                          0
redirectiontolte_coverageout           0
redirectiontolte_epsfallback           0
redirectiontolte_emergencyfallback     0
handoversucc                           0
reestabsucc                            0
rlculbyte                             18
rlcdlbyte                             18
totprbulavg                           18
totprbdlavg                           18
dlreceivedriavg                       18
dltransmittedmcsavg                   18
airmaculbyte                          18
airmacdlbyte                          18
bler_ul                               18
bler_dl                               18
numrar                                18
nummsg3                               18
attpaging       

In [123]:
imputer = SimpleImputer(strategy='mean')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

In [124]:
data.head()

,scgfail,scgfailratio,erabaddsucc,endcaddsucc,endcmodbymenbsucc,endcmodbysgnbsucc,connestabsucc,redirectiontolte_coverageout,redirectiontolte_epsfallback,redirectiontolte_emergencyfallback,...,nummsg3,attpaging,rssipathavg,dlreceivedcqiavg,endcrelbymenb,year,month,day,hour,minute
0,0.0,0.0,12.0,10.0,0.0,0.0,4.0,0.0,0.0,0.0,...,17.0,2243.0,-93.3,11.1,0.0,2023.0,5.0,19.0,0.0,0.0
1,0.0,0.0,28.0,27.0,11.0,33.0,1.0,0.0,0.0,0.0,...,39.0,2240.0,-93.5,10.2,4.0,2023.0,5.0,19.0,0.0,0.0
2,1.0,3.7,27.0,17.0,8.0,15.0,4.0,1.0,0.0,0.0,...,53.0,1337.0,-92.7,11.3,2.0,2023.0,5.0,19.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,...,5.0,1401.0,-92.8,13.6,0.0,2023.0,5.0,19.0,0.0,0.0
4,0.0,0.0,29.0,29.0,1.0,18.0,1.0,0.0,0.0,0.0,...,35.0,2083.0,-92.7,12.4,28.0,2023.0,5.0,19.0,0.0,0.0


In [125]:
print(data.shape)

(274890, 33)


In [126]:
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [127]:
data = np.log(data)
labels = np.log(labels)

data[data == -np.inf] = 0
data[data == np.inf] = 0
labels[labels == np.inf] = 0
labels[labels == -np.inf] = 0

C:\Users\INTEL WIN11PRO(12TH)\AppData\Local\Temp\ipykernel_24772\3420448403.py:1: RuntimeWarning: divide by zero encountered in log
  data = np.log(data)
C:\Users\INTEL WIN11PRO(12TH)\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [128]:
train = data[:137445]
test = data[137445:]

In [129]:
pca= PCA()
pca.fit(train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
dim = np.argmax(cumsum >= 0.90) + 1
print('The number of dimensions required to preserve 90% of variance is',dim)

The number of dimensions required to preserve 90% of variance is 11


In [177]:
pca_components = pca.components_
for i, component in enumerate(pca_components[:11]):
    print(f"Component {i+1}: {component}")

Component 1: [ 1.29373007e-01  1.12472876e-01 -1.13581068e-01 -9.14429377e-02
  1.16980784e-01 -1.26979957e-01 -3.04893466e-03  3.85162422e-02
  1.74296841e-02 -2.52435490e-29  4.91680292e-02  7.51024098e-03
 -3.45503337e-01 -5.00842575e-01  4.01993737e-01 -4.86335751e-02
 -1.11379769e-02 -7.99101045e-02 -3.26885872e-01 -4.88521538e-01
 -3.34915443e-02 -5.54809115e-03 -7.85520259e-02 -1.19859811e-01
 -3.11315596e-02 -9.75915756e-03 -2.66954573e-03  7.01822867e-02
  0.00000000e+00 -1.22198881e-03 -2.80892244e-04 -3.72236742e-02
  4.48132379e-04]
Component 2: [ 2.67719104e-01  1.93058283e-01 -6.20404448e-04  2.86704785e-02
  3.54407138e-01  8.92853990e-02  9.08015842e-02  4.05555862e-02
  1.68894704e-02 -0.00000000e+00  1.30410811e-01  4.61736875e-03
  4.80976344e-01  4.69498603e-01  3.56447458e-01 -1.26378871e-02
 -8.70262311e-03  5.81464829e-03 -1.71200850e-01 -1.47339916e-01
  3.19737846e-02  1.10093858e-01 -7.88237496e-02 -9.41696584e-02
 -6.98900905e-02 -2.58138634e-03 -5.42746498e-

In [130]:
pca = PCA(n_components=11, whiten=True)
pca = pca.fit(data)
dataPCA = pca.transform(data)

In [131]:
dataPCA.shape

(274890, 11)

In [132]:
train = dataPCA[:137445]
test = dataPCA[137445:]


In [133]:
train.shape

(137445, 11)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size = 0.3, random_state = 0)

In [136]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(96211, 11)
(41234, 11)
(96211,)
(41234,)


In [139]:
np.sqrt(np.mean((y_test-np.mean(y_test))**2))

0.6614419880478333

* 선형회귀

In [140]:
from sklearn.linear_model import LinearRegression

In [149]:
ols_regr = LinearRegression()
ols_regr.fit(X_train,y_train)
ols_pred = ols_regr.predict(X_test)
ols_pred[ols_pred<0,]=0
ols_pred
mae = mean_absolute_error(y_test, ols_pred)
print("Mean Absolute Error (MAE):", mae)

rmse = np.sqrt(np.mean((y_test - ols_pred)**2))
print("Root Mean Squared Error (RMSE):", rmse)

print("Coefficients:", ols_regr.coef_)

Mean Absolute Error (MAE): 0.2957945966514273
Root Mean Squared Error (RMSE): 0.40465096833027103
Coefficients: [-0.41746152 -0.23609246 -0.007226    0.01307302  0.01350826  0.06640663
  0.01997202 -0.14329921 -0.08033085 -0.05922293 -0.08061903]


In [147]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50): #find the best n_estimators
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='rmse', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #print the best n_estimators
    #print(cvresult)
    print("n_estimators:", cvresult.shape[0])

In [150]:
xgb_regr = xgb.XGBRegressor( #just arbitrary xgboost
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    nthread=3,
    scale_pos_weight=1,
    seed=1
    )

In [152]:
xgb_regr.fit(X_train, y_train)
xgb_pred = xgb_regr.predict(X_test)

In [155]:
mae = mean_absolute_error(y_test, xgb_pred)
print("Mean Absolute Error (MAE):", mae)

rmse = np.sqrt(np.mean((y_test - xgb_pred)**2))
print("Root Mean Squared Error (RMSE):", rmse)


Mean Absolute Error (MAE): 0.20986798838256204
Root Mean Squared Error (RMSE): 0.27440320426750786


In [160]:
xgb2_regr = xgb.XGBRegressor(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=13,
    min_child_weight=7,
    gamma=0,
    subsample=0.9,
    colsample_bytree=0.9,
    objective='reg:squarederror',
    nthread=3,
    scale_pos_weight=1,
    reg_alpha=50,
    reg_lambda=0.05,
    seed=1
    )

In [161]:
xgb_regr2.fit(X_train, y_train)
xgb_pred2 = xgb_regr2.predict(X_test)

In [162]:
mae = mean_absolute_error(y_test, xgb_pred2)
print("Mean Absolute Error (MAE):", mae)

rmse = np.sqrt(np.mean((y_test - xgb_pred2)**2))
print("Root Mean Squared Error (RMSE):", rmse)


Mean Absolute Error (MAE): 0.21960510657874827
Root Mean Squared Error (RMSE): 0.28728570239402196


In [ ]:
submission_data = pd.read_csv("./csv_file/Q1_label_sample.csv")

In [171]:
result_df = pd.DataFrame({
    'datetime': test_df['datetime'].iloc[:len(xgb_pred)],
    'BaseStationB': np.exp(xgb_pred),
    'BaseStationJ': np.exp(xgb_pred)
})

ValueError: array length 41234 does not match index length 137445

In [ ]:
result_df.to_csv("result.csv", index=True)